#Trabajo Final Grupo Nº 2

Rafael Perez	rafaelperezctes@gmail.com
Olinca Ayala	olincayalan88@gmail.com
###@Created on Dec Sat 2 16:57:20 2023
###@author: Grupo Nº 2


# Modulo Vision

Este cuaderno realiza la descarga y análisis de secuencias nucleotídicas de integronas de Shewanella utilizando Biopython.


In [ ]:
# @title Instalación de librerias
import subprocess
import ipywidgets as widgets
from IPython.display import display
import pkg_resources

def instalar_librerias():
    librerias = [
        "import_ipynb",
        "nbconvert",
        "ipywidgets",
        "biopython",
        "pandas",
        "numpy",
        "matplotlib",
        "tqdm",
        "odfpy",
        "requests",
        "gdown",
        "mafft",
        "clustalo",
        "muscle",
        "fasttree",
        "weblogo",
        "logomaker",

    ]

    barra_progreso = widgets.IntProgress(
        value=0,
        min=0,
        max=len(librerias),
        bar_style='info',
        orientation='horizontal'
    )

    etiqueta = widgets.Label(value='Iniciando instalación...')
    display(widgets.VBox([etiqueta, barra_progreso]))

    # Instalación de fasttree y mafft con apt-get (solo ejecutar en Colab)
    subprocess.run(["apt-get", "install", "fasttree", "mafft", "ghostscript"])

    for libreria in librerias:
        try:
            pkg_resources.require(libreria)
            etiqueta.value = f"{libreria} ya está instalado."
        except pkg_resources.DistributionNotFound:
            etiqueta.value = f"Instalando: {libreria}..."
            subprocess.run(["pip", "install", libreria])
        barra_progreso.value += 1

    etiqueta.value = 'Instalación completada.'
    # importo todas las librerias
    importarLibrerias()
# Crear botón
boton_instalar = widgets.Button(description='Instalar Librerías')

# Crear área de salida
output = widgets.Output()
display(output)

# Enlazar el botón con la función de instalación
boton_instalar.on_click(lambda b: instalar_librerias())

# Mostrar botón
#display(boton_instalar)

#instalar_librerias()


Output()

Button(description='Instalar Librerías', style=ButtonStyle())

In [ ]:
def main():
  instalar_librerias()
#main()

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Oct 19 14:17:44 2021

@author: android-2d
"""

from Bio.PDB.MMCIFParser import MMCIFParser
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB.MMCIF2Dict import MMCIF2Dict
import warnings
warnings.filterwarnings("ignore")

def LeerPDB(structure_id,filename):
    parser = PDBParser(PERMISSIVE=1)
    structure = parser.get_structure(structure_id, filename)
    return structure

def LeerCIF(filename):
    mmcif_dict = MMCIF2Dict(filename)
    for value,key in mmcif_dict.items():
        print(value,key)
    return mmcif_dict

def Graficar(structure):
    import nglview as nv
    view = nv.show_biopython(structure)
    view.render_image()
    view._display_image()
    view
    return view
    #view2.clear_representations()
    #view2.add_representation('cartoon', selection='protein', color='blue')
    #view2.add_representation('licorice', selection='not hydrogen')

def SelecionePDB(lista_pdb):
    ingreso = int(input("Ingrese el numero del PDB a graficar: "))
    if ingreso!=0:
        salida = lista_pdb[ingreso-1]
    else:
        salida=0
    return salida

def ListarPDB():
    import os
    import time
    actual_dir = os.getcwd()
    PDB_dir= actual_dir+"\\BD\\PDB\\"
    contenido = os.listdir(PDB_dir)
    c=1
    lista_pdb=[]
    print("Se han encotrado los siguientes archivos PDB:")
    for archivo in contenido:
        if archivo.endswith('.pdb'):
            print(c,":",archivo)
            c+=1
            lista_pdb.append(PDB_dir+archivo)

    time.sleep(1)
    pdb_elegido=SelecionePDB(lista_pdb)
    return pdb_elegido

In [ ]:
def main():
        structure_id = 'miPDB'
        filename = ListarPDB()
        structure = LeerPDB(structure_id,filename)
        view=Graficar(structure)
        return view

main()

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# # TP6 Grupo Nº 5
# Perez Ernesto Rafael rafaelperezctes@gmail.com Sofia Erdozain sofierdozain@gmail.com
# Created on Wed Nov  3 14:02:59 2021

# @author: Grupo Nº 5


import sys
import Alineamientos
import CargaBD
import Main as principal
import Utils

proteinas = CargaBD.proteinas
from Bio.Blast import NCBIWWW, NCBIXML
from Bio import SeqIO
from io import StringIO

def buscarBD(archivo):
    f_record = next(SeqIO.parse(archivo, "fasta"))
    # busco en la BD
    print("corriendo blast recuperando datos...")
    result_handle = NCBIWWW.qblast("blastp", "pdb", f_record.format("fasta"))
    # guardo resultados
    qblast_out = 'm_cold_blast.out'
    archivo_p = "BD/" + qblast_out
    with open(archivo_p, "w") as save_file:
        blast_results = result_handle.read()
        save_file.write(blast_results)
    result_handle.close()
    return blast_results

def AnalizarResultados(blast_results, nombre):
    print("Analizando los resultados y extrayendo información...")
    # opción 1 - abre el archivo guardado para analizarlo
    # opción 2 - crear un identificador a partir de la cadena y analizarlo
    string_result_handle = StringIO(blast_results)
    b_record = NCBIXML.read(string_result_handle)
    # ahora obtengo la información de alineación
    # para todos los valores e mayores que algún umbral
    E_VALUE_THRESH = 0.5
    lista_blast = []
    print("Se encontraron ", len(b_record.alignments), "estructuras asociadas a ", nombre)
    c = 1
    for alignment in b_record.alignments:
        for hsp in alignment.hsps:
            # if hsp.expect < E_VALUE_THRESH:
            print()
            print("Proteina ", c, " ID:", alignment.accession)
            lista_blast.append(str(alignment.accession))
            print("****Alineacion****")
            print("secuencia: %s" % alignment.title)
            print("longitud: %i" % alignment.length)
            print("e valor: %f" % hsp.expect)
            print(hsp.query[0:75] + "...")
            print(hsp.match[0:75] + "...")
            print(hsp.sbjct[0:75] + "...")
        c += 1
    return lista_blast


def SubMenu2(lista_blast):
    print()
    print("### Modulo: Estructuras NCBI ###")
    print()
    indice = int(input("Elija el numero de la estructura que desea descargar - 0 para salir: "))
    if indice == 0:
        SubMenu1()
    else:
        from Bio.PDB import PDBList
        pdbl = PDBList()
        lista_pdb = [lista_blast[indice - 1][0:4]]  # debo buscar solo las 4 letras iniciales
        pdbl.download_pdb_files(lista_pdb, obsolete=False, pdir="BD/PDB/", file_format="pdb", overwrite=True)
        # pdbl.retrieve_pdb_file('1ms5', file_format="pdb")
        # https://files.rcsb.org/download/1MS5.pdb


def SubMenu1():
    print("### Modulo: Estructuras NCBI ###")
    print()
    Utils.ListarProteinas()
    proteinas_lista = list(CargaBD.proteinas)
    indice = int(input("Elija el numero de la proteina para analizar: "))
    nombre = str(proteinas_lista[indice - 1])
    archivo = "BD/" + nombre + ".fasta"
    return archivo, nombre


def Visualizar():
    import subprocess
    from subprocess import PIPE
    print(
        "Se ejecutara Jupyter-netbook en su navegador y se abrira una consola (Cierre la consola de Jupyter para volver al menu )")
    comando = "jupyter notebook EstructurasNCBI-visualizador.ipynb"
    result = subprocess.Popen(comando)
    result.communicate()
    return 1


def Menu():
    print()
    print("### Modulo: Estructuras NCBI###")
    print()
    print("Que desea realizar:")
    print("1- Buscar estructura para Proteinas almacenadas")
    print("2- Visualizar proiteinas")
    print("3- Alinear (Beta)")
    print("4- Volver al menu principal")
    print("5- Salir")
    CargaBD.LeerInicio()
    opcion_principal = int(input("Ingrese una opción: "))
    if opcion_principal == 1:
        archivo, nombre = SubMenu1()
        blast_results = buscarBD(archivo)
        lista_blast = AnalizarResultados(blast_results, nombre)
        SubMenu2(lista_blast)
        Menu()
    if opcion_principal == 2:
        Visualizar()
        Menu()
    if opcion_principal == 3:
        Utils.ListarGenomas()
        Menu()
    if opcion_principal == 4:
        principal.Menu()
    if opcion_principal == 5:
        print("Gracias por utilizar BIOG5")
        sys.exit()
# Menu()